<a href="https://colab.research.google.com/github/devd-09/Gee_Projects/blob/main/WB_LST_gif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
import geemap,geemap as import_map
import ee
import os
ee.Authenticate()
ee.Initialize(project='debayan2002')
vis_map=import_map.Map()

In [251]:
wb_boundary=ee.FeatureCollection('projects/debayan2002/assets/wb_boundary')
vis_map.addLayer(wb_boundary,{},'WB Boundary')

In [252]:
region = ee.Geometry.Polygon(
    [[[85.30, 21.30], [90.10, 21.30], [90.10, 27.50], [86.30, 27.50],[85.30, 21.30]]],
    None,
    False)

In [281]:
LST_data=ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')

In [282]:
ndvi_india = LST_data.map(lambda img:img.set('doy',
                                                   ee.Date(img.get('system:time_start')).getRelative('day', 'year')))

In [283]:
doy=ndvi_india.filterDate('2021-01-31','2022-01-31')

In [284]:
filter=ee.Filter.equals(leftField='doy',rightField='doy')
join=ee.Join.saveAll('doy_matches')
joincol=ee.ImageCollection(join.apply(doy,ndvi_india,filter))

In [285]:
stack=joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()))


In [286]:
ndviVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
};

In [287]:
rgbvis=stack.map(lambda img:img.visualize(bands=['LST_Day_1km_mean'],**ndviVis).clip(wb_boundary))

In [288]:
gifparams={
    'region':region,
    'dimensions':304,
    'crs':'EPSG:4326',
    'framesPerSecond':10,
    'format':'gif'
}

In [289]:
print(rgbvis.getVideoThumbURL(gifparams))

https://earthengine.googleapis.com/v1/projects/debayan2002/videoThumbnails/c77c13048933759889f1f5daaccd199a-28974446a168f22d92de7b3086622396:getPixels
